<img align="left" src="https://quantumspain-project.es/wp-content/uploads/2022/11/Logo_QS_EspanaDigital.png" width="1000px"/><br><br><br><br>


# QML (Quantum Machine Learning)

Created: 2022/10/30

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img aling="left" alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />License: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Licencia Creative Commons Atribución-CompartirIgual 4.0 Internacional</a>.
Internal Reviewers:
* Alba Cervera ([BSC](https://www.bsc.es/))

Authors:
* Carmen Calvo ([SCAYLE](https://www.scayle.es/))
* Antoni Alou ([PIC](https://www.pic.es/))
* Carlos Hernani ([UV](https://www.uv.es/))
* Nahia Iriarte ([NASERTIC](https://www.nasertic.es/es))
* Carlos Luque ([IAC](https://www.iac.es/))


Consulta la notación que se ha utilizado durante todo el documento en el siguiente [enlace](#notacion).

# 3. Variational Quantum Classifier (VQC)

Existen múltiples métodos para clasificar un conjunto de datos utilizando un ordenador cuántico, uno de ellos es el algoritmo conocido como ***Variational Quantum Classifier*** o **VQC**.

El VQC es un algoritmo de aprendizaje supervisado que hace uso de un circuito cuántico variacional. El circuito cuántico variacional, también llamado circuito cuántico parametrizado, realiza una función similar a la arquitectura de una red neuronal clásica.

Al igual que el aprendizaje automático clásico, el algoritmo VQC tiene una fase de entrenamiento, en la que se proporcionan datos etiquetados, y una fase de validación, en la que se clasifican nuevos datos con el modelo ya entrenado para así evaluar la calidad del modelo [[1]](#referencias). Se trata de un algoritmo de clasificación híbrido ya que combina una componente cuántica y una componente clásica, tal y como se aprecia en la imagen 1, que realiza un bucle de optimización de los parámetros del circuito cuántico. 

Comienza con la componente cuántica, en esta fase se lleva a cabo el preprocesamiento de los datos, la preparación de los estados cuánticos y se ejecuta el circuito correspondiente al algoritmo. Una vez se completa la primera fase, el algoritmo hace uso de la componente clásica. Esta última, ajusta la función de coste y actualiza los parámetros en función de la salida obtenida en el circuito.

<center><img src=https://raw.githubusercontent.com/born-2learn/born-2learn.github.io/master/_posts/images/vqc-part1/qml-workflow.png width="400"></center>

<center>Imagen 1. Etapas algoritmo QML [5]</center>

Ambas fases se llevan a cabo de forma iterativa, es decir, los valores de los parámetros se actualizan con los obtenidos en la componente clásica y se iniciará de nuevo el proceso con la finalidad de obtener la mejor solución. 

<a id='VariationalQuantumCirc'></a>
## 3.1. Variational Quantum Circuit

Los circuitos cuánticos variacionales o parametrizados son circuitos que dependen de parámetros configurables que se optimizan mediante un coprocesador clásico para la realización de una tarea determinada.

El principio de funcionamiento de los circuitos variacionales es muy similar al de las redes neuronales clásicas, por lo que desempeñan un papel importante en el aprendizaje automático en ordenadores cuánticos.

En general, todo circuito variacional se divide en tres pasos:

1. **Preparación de datos (*State Preparation*)**: Los datos clásicos necesitan ser trasladados a estados cuánticos en un espacio de Hilbert mediante un mapeado de características cuántico (***Quantum Feature Map***). A este paso se le conoce también como ***quantum embedding*** o ***data encoding***. Para conocer más a cerca de estas técnicas puede consultar el siguiente [*enlace*](2_Feature_encoding.ipynb).


2. **Circuito parametrizado**: El circuito diseñado para realizar una tarea concreta, por ejemplo clasificación, está caracterizado por un operador unitario parametrizado $U(x, \theta)$. Siendo $x$ la entrada del circuito (los datos anteriormente codificados) y $\theta$ los parámetros variacionales. 


3. **Medida/Optimización**: En esta etapa se produce la medida de un observable a la salida del circuito. El circuito del paso anterior se ejecuta conjuntamente con un coprocesador clásico que realiza el cálculo de la función de coste a partir del valor esperado del observable medido y la actualización de los parámetros variacionales a partir de la salida del circuito [[2]](#referencias).


<a id='Ejemplo'></a>
## 3.2. Ejemplo de aplicación

A continuación, se recoge un ejemplo básico con el que se demuestra el uso de un clasificador cuántico variacional para reproducir la función de paridad


$$
\begin{align}
&f: \{0,1\}^n \rightarrow \{0, 1\}\\
&f(x) = x_1 \oplus x_2 \oplus \dots \oplus x_n
\end{align}
$$


tal que $f(x) = 1$ si y solo si el número de unos en $x$ es impar y $f(x)=0$ en el otro caso.

Este ejemplo de optimización demuestra cómo codificar entradas binarias en el estado inicial del circuito variacional [[3]](#referencias). 

El clasificador está escrito en Python usando la librería PennyLane, un *framework* para programación cuántica similar a los utilizados en la computación clásica, Tensorflow o PyTorch. PennyLane está desarrollado en base a la idea de entrenar circuitos cuánticos utilizando la diferenciación automática. Esto es especialmente importante en aplicaciones como el aprendizaje automático cuántico, la química cuántica y la optimización cuántica [[4]](#referencias).


Primero se instala la librería PennyLane

In [1]:
!pip install pennylane

Una vez instalada, se carga la librería y las herramientas necesarias.

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

El problema XOR de 4 bits se puede codificar mediante el *basis embedding* usando cuatro qubits.

In [3]:
n_qubits = 4
def statepreparation(x:list):
    # expects a list of 0 and 1
    # a way to encode data inputs x into the circuit
    qml.BasisState(x, wires=list(range(n_qubits)))

Se define una capa del circuito mediante rotaciones arbitrarias [Rot](https://docs.pennylane.ai/en/stable/code/api/pennylane.Rot.html#pennylane.Rot) y puertas [CNOT](https://docs.pennylane.ai/en/stable/code/api/pennylane.CNOT.html). 

In [4]:
dev = qml.device("default.qubit", wires=n_qubits)

def layer(weight):
    # arbitrary rotation on every qubit
    qml.Rot(weight[0, 0], weight[0, 1], weight[0, 2], wires=0)
    qml.Rot(weight[1, 0], weight[1, 1], weight[1, 2], wires=1)
    qml.Rot(weight[2, 0], weight[2, 1], weight[2, 2], wires=2)
    qml.Rot(weight[3, 0], weight[3, 1], weight[3, 2], wires=3)
    #CNOTs to entangle each qubit with the neighbor next to it
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

Se define el circuito variacional como la unión de las tres etapas mencionadas anteriormente : 

- Preparación del estado.
- Circuito.
- Medida.

In [5]:
@qml.qnode(dev)
def circuit(weights, x):
    statepreparation(x)
    for weight in weights:
        layer(weight)
    return qml.expval(qml.PauliZ(0))

Se define el clasificador (*variational quantum classifier*) como el circuito anterior y un término clásico, *bias*.

In [6]:
def variational_classifier(weights, bias, x):
    # classical bias parameter
    return circuit(weights, x) + bias

Se definen dos funciones de coste distintas, *square loss* y *accuracy*, a continuación se adjunta la expresión matemática de cada una de ellas. 



$$
\mathit{\text{Square loss}} = \frac {\sum_{i=1}^N (y_i - \tilde{y_i})^2}{N} ~~~~~~~~~~
\mathit{\text{Accuracy}} = \frac{\text{Predicciones correctas}}{N}
$$

In [7]:
def sqloss_acc(labels, predictions):
    sqloss = 0
    acc = 0
    for label, prediction in zip(labels, predictions):
        sqloss = sqloss + (label - prediction)**2
        if abs(label-prediction) < 1e-5:
            acc += 1
    sqloss = sqloss / len(labels)
    acc = acc / len(labels)
    return sqloss, acc

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return sqloss_acc(Y, predictions)[0]

Se generan los datos y los parámetros iniciales del circuito

In [8]:
def parity_data():
    data = np.array([
        [0,0,0,0,0],
        [0,0,0,1,1],
        [0,0,1,0,1],
        [0,0,1,1,0],
        [0,1,0,0,1],
        [0,1,0,1,0],
        [0,1,1,0,0],
        [0,1,1,1,1],
        [1,0,0,0,1],
        [1,0,0,1,0],
        [1,0,1,0,0],
        [1,0,1,1,1],
        [1,1,0,0,0],
        [1,1,0,1,1],
        [1,1,1,0,1],
        [1,1,1,1,0],
    ], requires_grad=False)
    X = np.array(data[:, :-1], requires_grad=False)
    Y = np.array(data[:, -1], requires_grad=False)
    Y = Y * 2 - np.ones(len(Y))  # shift label from {0, 1} to {-1, 1}
    return X, Y

X, Y = parity_data()
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]] 0.0


Así como los hiperparámetros del entrenamiento y se ejecuta el clasifcador.

In [9]:
n_iter = 25
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

weights = weights_init
bias = bias_init
for it in range(n_iter):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    # the optimizer needs a callable cost function
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    cost_, acc = sqloss_acc(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost_, acc
        )
    )



Iter:     1 | Cost: 2.0000000 | Accuracy: 0.5000000 
Iter:     2 | Cost: 2.0000000 | Accuracy: 0.5000000 
Iter:     3 | Cost: 2.0000000 | Accuracy: 0.5000000 
Iter:     4 | Cost: 2.0000000 | Accuracy: 0.5000000 
Iter:     5 | Cost: 2.0000000 | Accuracy: 0.5000000 
Iter:     6 | Cost: 1.5000000 | Accuracy: 0.6250000 
Iter:     7 | Cost: 2.0000000 | Accuracy: 0.5000000 
Iter:     8 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:     9 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    10 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    11 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    12 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    13 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    14 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    15 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    16 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    17 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    18 | Cost: 0.0000000 | Accuracy: 1.0000000 
Iter:    19 | Cost: 0.0000000 | Accuracy: 1.00

Como se puede observar a partir de la iteración $8$, la métrica *accuracy* es $1.0$ i.e. clasifica perfectamente el problema. A continuación, una visualización del circuito variacional.

In [10]:
drawer = qml.draw(circuit)
print(drawer(weights, X))

0: ─╭BasisState(M0)──Rot(0.02,1.41,-0.00)──╭●───────╭X──Rot(0.01,0.00,0.00)───╭●───────╭X─┤  <Z>
1: ─├BasisState(M0)──Rot(0.02,1.43,-0.00)──╰X─╭●────│───Rot(-0.32,0.03,-0.00)─╰X─╭●────│──┤     
2: ─├BasisState(M0)──Rot(0.01,-0.04,-0.06)────╰X─╭●─│───Rot(0.10,0.06,-0.03)─────╰X─╭●─│──┤     
3: ─╰BasisState(M0)──Rot(0.00,1.57,0.01)─────────╰X─╰●──Rot(-0.00,1.57,-0.01)───────╰X─╰●─┤     


<a id='notacion'></a>
<div class="alert alert-block alert-warning",text-align:center>
<b>ANEXO NOTACIÓN:</b>

Para que la comprensión de los notebooks sea mejor se ha unificado la notación utilizada en los mismos. Para diferenciar un vector de un valor único se hará uso de la negrita. De manera que $\mathbf{x}$ corresponde a un vector y $z$ será una variable de una única componente. 

    
Si se quiere hacer referencia a dos vectores distintos pero que pertenecen al mismo *dataset* se utilizará un subíndice, es decir, $\mathbf{x_i}$ hará referencia al i-ésimo vector del dataset. Si se quiere referenciar una característica concreta del vector $\mathbf{x_i}$ se añadirá un nuevo subíndice, de manera que $\mathbf{x_{i_j}}$ hará referencia a la j-ésima variable del i-ésimo vector.

</div>

---------------------------
## Referencias
<a id='referencias'></a>
[1]. https://born-2learn.github.io/posts/2020/12/variational-quantum-classifier/ <br>
[2]. https://pennylane.ai/qml/glossary/variational_circuit.html <br>
[3]. https://pennylane.ai/qml/demos/tutorial_variational_classifier.html <br>
[4]. https://pennylane.ai/faq.html <br>
[5]. https://blog.tensorflow.org/2020/03/announcing-tensorflow-quantum-open.html<br>

This work has been financially supported by the Ministry of Economic Affairs and Digital Transformation of the Spanish Government through the QUANTUM ENIA project call - Quantum Spain project, and by the European Union through the Recovery, Transformation and Resilience Plan - NextGenerationEU within the framework of the Digital Spain 2025 Agenda.


<img align="left" src="https://quantumspain-project.es/wp-content/uploads/2022/11/LOGOS-GOB_QS.png" width="1000px" />